# DATENMANIPULATION
#### VORSICHT HIER IST VIEL GEPFUSCHT

In [1]:
import warnings
import itertools
import numpy as np

from glob import glob
from matplotlib import pyplot as plt
from mastermetry import Master
from convert import RA2sec, sec2RA, DEC2arcsec, arcsec2DEC, open_map
from numerik import Regression

warnings.filterwarnings("ignore")

In [2]:
# some global variables
GAIN = 1.56
res = [0, 2048]
dpi = 80
figsize = np.array((res[1]/float(dpi), res[1]/float(dpi)))
ticks = np.linspace(*res, 9, dtype=int)
rF = "V"  # prime filter

In [3]:
master = Master(
    glob(r"./20190217/*.fit"),
    gain=GAIN,
    method="median"
)

In [4]:
master.export_Tables(
    "./map_%s.csv",
#   method="median",
    sigma=3,
    fwhm=4,
    kappa=10,
    r=3,
    r_in=5,
    r_out=9,
    overwrite=True
)

In [5]:
rmap = open_map("./map_ref.csv")
fmaps = {
    "R": open_map("./map_R.csv"),
    "V": open_map("./map_V.csv"),
    "B": open_map("./map_B.csv")
}

In [6]:
# determine precise reference coordinates
n = len(rmap["ID"])
rfmaps = {
    "R": np.zeros((2, n), dtype=np.float32),
    "V": np.zeros((2, n), dtype=np.float32),
    "B": np.zeros((2, n), dtype=np.float32)
}
idfmaps = {
    "R": {},
    "V": {},
    "B": {}
}
# set of id's used to avoid double marks
idref = {
    "R": set(),
    "V": set(),
    "B": set()
}
for ftype, fmap in fmaps.items():
    for i, rid in enumerate(rmap["ID"]):
        min_diff = 999
        min_idx = 0
        for j in range(len(fmap["id"])):
            # look where the differenze of the "eye-obtained" pixel values minus the calculated values is the smallest
            diff = np.mean([abs(rmap["X"][i] - fmap["xcenter"][j]), abs(rmap["Y"][i]  - fmap["ycenter"][j])])
            if diff < min_diff:
                min_idx = j
                min_diff = diff
        idref[ftype].add(fmap["id"][min_idx])
        idfmaps[ftype][rid] = fmap["id"][min_idx]
        rfmaps[ftype][0, i] = fmap["xcenter"][min_idx]
        rfmaps[ftype][1, i] = fmap["ycenter"][min_idx]
for key, val in idfmaps.items():
    print(key, val)

R {1: 255, 2: 248, 3: 204, 4: 174, 5: 153, 6: 134, 7: 95, 8: 83, 9: 60}
V {1: 169, 2: 167, 3: 142, 4: 121, 5: 107, 6: 92, 7: 65, 8: 56, 9: 42}
B {1: 127, 2: 125, 3: 105, 4: 90, 5: 78, 6: 65, 7: 45, 8: 40, 9: 28}


In [7]:
plt.rcParams['text.latex.preamble'] = [r"\usepackage{amsmath}\usepackage{siunitx}"]
plt.rc("text", usetex=True)
plt.rc("font", family="serif", size=30)
fig, ax = plt.subplots(figsize=figsize/2)
X = rfmaps[rF][0]
RA = np.array(rmap["RA"], dtype=np.float32)
XRAreg = Regression(X, RA, [0, 1])
Xref = np.linspace(np.min(X), np.max(X), 100)
ax.scatter(
    X,
    RA,
    label="reference stars",
    color="black",
    marker="*",
    s=600,
    zorder=10
)
ax.plot(
    Xref,
    XRAreg.f(Xref),
    label="linear fit",
    color="red",
    linewidth=4,
    zorder=4
)
plt.title(r"$\text{RA} = \SI{%g}{\sec} + \SI{%g}{\sec} \cdot \text{X}$" % (XRAreg.alpha[0], XRAreg.alpha[1]))
plt.xticks(ticks, ticks)
RAticks = np.linspace(np.min(RA), np.max(RA), 9)
plt.yticks(RAticks, [sec2RA(t) for t in RAticks])
plt.xlim(*res)
plt.xlabel(r"X [pixel]")
plt.ylabel(r"RA [hh:mm:ss]")
plt.grid(
    linestyle="--",
    color="lightgrey",
    zorder=1
)
plt.legend()
fig.savefig("./regression_x_RA.pdf", bbox_inches="tight")
#plt.show()
plt.close()

In [8]:
plt.rcParams['text.latex.preamble'] = [r"\usepackage{amsmath}\usepackage{siunitx}"]
plt.rc("text", usetex=True)
plt.rc("font", family="serif", size=30)
fig, ax = plt.subplots(figsize=figsize/2)
Y = rfmaps[rF][1]
DEC = np.array(rmap["DEC"], dtype=np.float32)
YDECreg = Regression(Y, DEC, [0, 1])
Yref = np.linspace(np.min(Y), np.max(Y), 100)
ax.scatter(
    Y,
    DEC,
    label="reference stars",
    color="black",
    marker="*",
    s=600,
    zorder=10
)
ax.plot(
    Yref,
    YDECreg.f(Yref),
    label="linear fit",
    color="green",
    linewidth=4,
    zorder=4
)
plt.title(r"$\text{DEC} = \SI{%g}{\arcsecond} + \SI{%g}{\arcsecond} \cdot \text{Y}$" % (YDECreg.alpha[0], YDECreg.alpha[1]))
plt.xticks(ticks, ticks)
DECticks = np.linspace(np.min(DEC), np.max(DEC), 9)
plt.yticks(DECticks, [arcsec2DEC(t) for t in DECticks])
plt.xlim(*res)
plt.xlabel(r"Y [pixel]")
plt.ylabel(r"DEC [dd:mm:ss]")
plt.grid(
    linestyle="--",
    color="lightgrey",
    zorder=1
)
plt.legend()
fig.savefig("./regression_y_DEC.pdf", bbox_inches="tight")
#plt.show()
plt.close()

In [9]:
# find min/max mag for each filter
mmap = {
    "R": {"X": [], "Y": [], "L": [], "F": []},
    "V": {"X": [], "Y": [], "L": [], "F": []},
    "B": {"X": [], "Y": [], "L": [], "F": []},
}
for F in "RVB":
    max_mag = -99
    min_mag = 99
    max_i = 0
    min_i = 0
    for i, mag in enumerate(fmaps[F]["mag"]):
        if mag > max_mag:
            max_mag = mag
            max_i = i
        if mag < min_mag:
            min_mag = mag
            min_i = i
    mmap[F]["X"].append(fmaps[F]["xcenter"][max_i])
    mmap[F]["Y"].append(fmaps[F]["ycenter"][max_i])
    mmap[F]["F"].append(fmaps[F]["residual_aperture_sum"][max_i])
    mmap[F]["L"].append(max_mag)
    mmap[F]["X"].append(fmaps[F]["xcenter"][min_i])
    mmap[F]["Y"].append(fmaps[F]["ycenter"][min_i])
    mmap[F]["L"].append(min_mag)
    mmap[F]["F"].append(fmaps[F]["residual_aperture_sum"][min_i])
for key, val in mmap.items():
    print(key, val)

R {'X': [1451.1355955665272, 998.4043998895172], 'Y': [1070.8689649051528, 928.6913978105746], 'L': [-0.5133841633225059, -8.712559830713285], 'F': [1.6045515165253228, 3055.089462713871]}
V {'X': [1087.8905558731844, 1525.6415097948066], 'Y': [1796.619574979341, 895.1947599595924], 'L': [0.11019098786192233, -8.74445810470124], 'F': [0.9034905300474421, 3146.177652274723]}
B {'X': [1850.8842570451743, 1524.467759742958], 'Y': [2007.1034502868854, 894.000939673071], 'L': [-1.7878883611116647, -8.901832279427254], 'F': [5.189856436805133, 3636.912995962797]}


In [10]:
# find all other stars
nmap = {
    "R": {"X": [], "Y": [], "ID": [], "L": [], "mag": [], "F": []},
    "V": {"X": [], "Y": [], "ID": [], "L": [], "mag": [], "F": []},
    "B": {"X": [], "Y": [], "ID": [], "L": [], "mag": [], "F": []}
}
T = [7, 7, 7]
for j, F in enumerate("RVB"):
    for i, mag in enumerate(fmaps[F]["mag"]):
        if str(mag) == "nan":
            continue
        elif not ((mag < 0) and (abs(mag) > T[j])):
            continue
        elif fmaps[F]["id"][i] not in idref[F]:
            nmap[F]["X"].append(fmaps[F]["xcenter"][i])
            nmap[F]["Y"].append(fmaps[F]["ycenter"][i])
            nmap[F]["ID"].append(fmaps[F]["id"][i])
            nmap[F]["mag"].append(mag)
            nmap[F]["F"].append(fmaps[F]["residual_aperture_sum"][i])
    print(len(nmap[F]["ID"]))
    for i in range(65, 91):
        nmap[F]["L"].append(chr(i))

13
12
13


In [11]:
# generate color offset permutations
offsets = {}
method = lambda P: int(round(np.mean(P), 0))
for perm in itertools.permutations([*rfmaps.keys()], r=2):
    offsets["{}-{}".format(*perm)] = [method(rfmaps[perm[0]][0] - rfmaps[perm[1]][0]), method(rfmaps[perm[0]][1] - rfmaps[perm[1]][1])]
print(offsets)

{'R-V': [-1, -1], 'R-B': [0, 0], 'V-R': [1, 1], 'V-B': [1, 1], 'B-R': [0, 0], 'B-V': [-1, -1]}


In [12]:
# apply color offset to the main object, use V filter as "point zero"
master.offset("R", *offsets["V-R"])
master.offset("B", *offsets["V-R"])
print(master.offset_history)
RGB = master.export_RGB("RVB")
resolution = RGB.shape

[(1, 1), (1, 1)]


In [13]:
# remove strange bright (hot?) pixels after the color offset
R = 1
M = 180
L = 90
for i in range(resolution[0]):
    for j in range(resolution[1]):
        for k in range(resolution[2]):
            if (RGB[i,j,k] >= M) and ((R <= i <= resolution[0]-R) and (R <= j <= resolution[1]-R)):
                # if the central pixel is 2 times as bright as the RxR mean around it, set it to half the mean
                RxR = RGB[i-R:i+R+1,j-R:j+R+1,k]
                Rmean = np.mean(RxR)
                if Rmean <= L:
                    print(i, j, k)
                    print(Rmean)
                    print(RxR)
                    print()
                    RGB[i,j,k] = Rmean / 2

445 404 2
72.0
[[ 48  70  44]
 [ 51 255  50]
 [ 43  43  44]]

714 1070 2
60.77777777777778
[[ 41  45  44]
 [ 45 194  45]
 [ 43  46  44]]

1611 1291 1
57.77777777777778
[[ 44  43  44]
 [ 44 204  43]
 [ 42  42  14]]

1612 1292 2
60.55555555555556
[[ 24  44  42]
 [ 44 216  45]
 [ 43  44  43]]

1685 883 0
87.22222222222223
[[ 49  73  60]
 [ 60 226 120]
 [ 50  84  63]]

1730 125 1
56.77777777777778
[[ 43  43  43]
 [ 43 196  42]
 [ 44  43  14]]

1731 126 2
61.44444444444444
[[ 25  43  41]
 [ 42 234  42]
 [ 42  43  41]]

1749 1039 1
55.333333333333336
[[ 42  44  42]
 [ 42 182  43]
 [ 43  43  17]]

1750 1040 2
58.666666666666664
[[ 26  43  43]
 [ 44 197  44]
 [ 44  44  43]]

1754 910 1
58.77777777777778
[[ 42  42  42]
 [ 43 218  43]
 [ 43  42  14]]

1755 911 2
60.666666666666664
[[ 22  44  43]
 [ 43 221  44]
 [ 42  44  43]]

1938 1259 1
61.77777777777778
[[ 42  43  43]
 [ 44 255  43]
 [ 43  43   0]]

1984 146 1
61.888888888888886
[[ 43  44  43]
 [ 42 237  41]
 [ 43  44  20]]



In [14]:
# "pretty picture"
# https://stackoverflow.com/questions/34768717/matplotlib-unable-to-save-image-in-same-resolution-as-original-image
fig = plt.figure(figsize=figsize)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
ax.imshow(RGB, interpolation='nearest')
ax.set(xlim=res[::-1], ylim=res, aspect=1)
fig.savefig("./pretty_picture.png", dpi=dpi, transparent=True)
#plt.show()
plt.close()

In [15]:
plt.rcParams['text.latex.preamble'] = [r"\usepackage{amsmath}\usepackage{siunitx}"]
plt.rc("text", usetex=True)
plt.rc("font", family="serif", size=30)
rF = "V"
rC = "yellow"
S = 600
fig, ax = plt.subplots(figsize=figsize)
ax.imshow(RGB)
for F in "RVB":
    if F == "R":
        color = "red"
        marker = "s"
        order = 2
    elif F == "V":
        color = "lime"
        marker = "8"
        order = 6
    elif F == "B":
        color = "dodgerblue"
        marker = "D"
        order = 4
    ax.scatter(rfmaps[F][0], rfmaps[F][1], s=S, facecolors="none", marker=marker, edgecolors=color, zorder=order)
    ax.scatter(nmap[F]["X"], nmap[F]["Y"], s=S, facecolors="none", marker=marker, edgecolors=color, zorder=order)
#   ax.scatter(mmap[F]["X"][0], mmap[F]["Y"][0], s=S, facecolors="none", marker=marker, edgecolors=color)
#   ax.annotate(str(r"$%+.2f\,\si{mag}$" % mmap[F]["L"][0]), (mmap[F]["X"][0], mmap[F]["Y"][0]),
#               xytext=(mmap[F]["X"][0]-20, mmap[F]["Y"][0]-40), color=color)
for i in range(len(rfmaps[rF][0])):
    ax.annotate(i+1, (rfmaps[rF][0][i], rfmaps[rF][1][i]), xytext=(rfmaps[rF][0][i]-20, rfmaps[rF][1][i]+20),
                color=rC, zorder=10)
for i in range(len(nmap[rF]["ID"])):
    ax.annotate(nmap[rF]["L"][i], (nmap[rF]["X"][i], nmap[rF]["Y"][i]),
                xytext=(nmap[rF]["X"][i]-20, nmap[rF]["Y"][i]+20), color=rC, zorder=10)
ax.scatter(mmap[rF]["X"][0], mmap[rF]["Y"][0], s=S, facecolors="none", marker="8", edgecolors="lime", zorder=6)
ax.annotate(str(r"$%+.2f\,\si{mag}$" % mmap[rF]["L"][0]), (mmap[rF]["X"][0], mmap[rF]["Y"][0]),
            xytext=(mmap[rF]["X"][0]-20, mmap[rF]["Y"][0]+20), color="lime", zorder=10)
ticks = np.linspace(*res, 9, dtype=int)
plt.xticks(ticks, ticks)
plt.yticks(ticks, ticks)
plt.xlim(*res[::-1])
plt.ylim(*res)
plt.xlabel(r"X$~[\,\si{pixel}\,]$")
plt.ylabel(r"Y$~[\,\si{pixel}\,]$")
plt.setp(ax.xaxis.get_majorticklabels(), ha="left")
plt.setp(ax.yaxis.get_majorticklabels(), va="bottom")
fig.savefig("./marked_map.png", bbox_inches="tight")
#plt.show()
plt.close()

In [16]:
tex = ""
for rid in rmap["ID"]:
    tex += r"%3d & %s & %s & %+6.2f & %+6.2f & %+6.2f  \\" % (
        round(rid, 0),
        sec2RA(rmap["RA"][rid-1]),
        arcsec2DEC(rmap["DEC"][rid-1]),
        rmap["B"][rid-1],
        rmap["V"][rid-1],
        rmap["R"][rid-1]
    )
    tex += "\n"
tex += "\n"
for rid in rmap["ID"]:
    tex += r"%3d & %4d & %4d & %+6.2f & %+6.2f & %+6.2f & %5d & %5d & %5d \\" % (
        round(rid, 0),
        round(fmaps[rF]["xcenter"][idfmaps[rF][rid]-1], 0),
        round(fmaps[rF]["ycenter"][idfmaps[rF][rid]-1], 0),
        float(fmaps["B"]["mag"][idfmaps["B"][rid]-1]),
        float(fmaps["V"]["mag"][idfmaps["V"][rid]-1]),
        float(fmaps["R"]["mag"][idfmaps["R"][rid]-1]),
        float(fmaps["B"]["residual_aperture_sum"][idfmaps["B"][rid]-1]),
        float(fmaps["V"]["residual_aperture_sum"][idfmaps["V"][rid]-1]),
        float(fmaps["R"]["residual_aperture_sum"][idfmaps["R"][rid]-1])
    )
    tex += "\n"
tex += "\n"
for i, rid in enumerate(nmap[rF]["ID"]):
    tex += r"%3s & %4d & %4d & %s & %s & %+6.2f & %+6.2f & %+6.2f & %5d & %5d & %5d \\" % (
        nmap[rF]["L"][i],
        round(nmap[rF]["X"][i], 0),
        round(nmap[rF]["Y"][i]),
        sec2RA(XRAreg.f(nmap[rF]["X"][i])),
        arcsec2DEC(YDECreg.f(nmap[rF]["Y"][i])),
        nmap["B"]["mag"][i],
        nmap["V"]["mag"][i],
        nmap["R"]["mag"][i],
        float(nmap["B"]["F"][i]),
        float(nmap["V"]["F"][i]),
        float(nmap["R"]["F"][i])
    )
    tex += "\n"
with open("tex_tables.txt", "w") as f:
    f.write(tex)
print(tex)

  1 & 06:48:48.52 & +41:11:28.4 & +13.96 & +12.71 & +12.35  \\
  2 & 06:47:55.57 & +41:11:10.0 & +13.63 & +12.92 & +12.75  \\
  3 & 06:48:22.51 & +41:08:03.9 & +15.63 & +14.53 & +14.12  \\
  4 & 06:48:37.54 & +41:06:21.1 & +10.38 & +10.14 & +10.21  \\
  5 & 06:47:47.56 & +41:05:08.7 & +14.26 & +13.60 & +13.44  \\
  6 & 06:48:55.36 & +41:03:49.3 & +11.97 & +11.40 & +11.30  \\
  7 & 06:48:17.94 & +41:01:15.9 & +14.48 & +13.76 & +13.55  \\
  8 & 06:48:57.06 & +41:00:25.1 & +14.25 & +13.49 & +13.28  \\
  9 & 06:48:10.05 & +40:59:10.1 & +11.39 & +10.92 & +10.85  \\

  1 & 1671 & 1707 &  -4.16 &  -4.63 &  -5.15 &    45 &    71 &   115 \\
  2 &  593 & 1660 &  -4.32 &  -4.37 &  -4.68 &    53 &    55 &    74 \\
  3 & 1146 & 1331 &  -2.32 &  -2.76 &  -3.33 &     8 &    12 &    21 \\
  4 & 1455 & 1150 &  -7.54 &  -7.27 &  -7.28 &  1041 &   811 &   818 \\
  5 &  437 & 1006 &  -3.67 &  -3.69 &  -3.98 &    29 &    29 &    38 \\
  6 & 1822 &  881 &  -6.03 &  -6.02 &  -6.18 &   258 &   254 &   297 \\
